In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn import preprocessing
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
#folders:
# ACSDT5Y2018.B25070_2020-11-19T202042 -- rent
# ACSST5Y2018.S1901_2020-11-20T153806 -- income
# 18zp33ny -- cares-exclusion
# ACSDP5Y2018.DP03_data_with_overlays_2020-11-20T094755 -- cares-exclusion
# ZIP_TRACT_122018 -- cares-exclusion

### Indicator 1 - Renters who pay more than 50% of their income on housing

In [3]:
rent = pd.read_csv('../data/rent-as-percentage-income/ACSDT5Y2018.B25070_data_with_overlays_2020-11-19T201947.csv',
                   low_memory=False, skiprows=[1],
                   usecols=['NAME','B25070_010E','B25070_001E','B25070_011E'])
rent.rename(columns={'B25070_010E': 'paying_50p_more', 'B25070_001E': 'Total',
                    'B25070_011E': 'Total_not_comp'}, inplace = True)

rent.head()

,NAME,Total,paying_50p_more,Total_not_comp
0,"Census Tract 37, New York County, New York",789,127,61
1,"Census Tract 86.01, New York County, New York",1001,204,49
2,"Census Tract 44, New York County, New York",8145,1439,224
3,"Census Tract 86.03, New York County, New York",736,204,16
4,"Census Tract 86.02, New York County, New York",0,0,0


In [4]:
# Extract County names
rent['County'] = rent['NAME'].str.split(pat = ',',expand = True)[1]
rent.drop(['NAME'], axis=1, inplace = True)

rent

,Total,paying_50p_more,Total_not_comp,County
0,789,127,61,New York County
1,1001,204,49,New York County
2,8145,1439,224,New York County
3,736,204,16,New York County
4,0,0,0,New York County
5,1999,252,55,New York County
6,3679,1038,172,New York County
7,371,113,18,New York County
8,2901,731,76,New York County
9,1303,248,40,New York County


In [5]:
# aggregate over counties
rent = rent.groupby(['County']).sum()

In [6]:
rent

,Total,paying_50p_more,Total_not_comp
County,,,
Bronx County,401745,132931,19630
Kings County,665526,193135,33830
New York County,575184,123027,25813
Queens County,431495,120943,24547
Richmond County,50313,14686,5613


In [7]:
rent['pay over 50'] = (rent['paying_50p_more'] / (rent['Total'] - rent['Total_not_comp'])).round(decimals=2)
rent

,Total,paying_50p_more,Total_not_comp,pay over 50
County,,,,
Bronx County,401745,132931,19630,0.35
Kings County,665526,193135,33830,0.31
New York County,575184,123027,25813,0.22
Queens County,431495,120943,24547,0.30
Richmond County,50313,14686,5613,0.33


In [8]:
# # rows with NaN - because has 0's for the columns being used for calculations
# rent[rent.isna().any(axis=1)]

# # rent table without  NaN 

# rent.dropna()

### Indicator 2 - Portion of low-income renters who have too little left ($12K) after paying housing cost

In [9]:
income = pd.read_csv('../data/income/ACSST5Y2018.S1901_data_with_overlays_2020-11-20T134203.csv',
                   low_memory=False)
income.head()

,GEO_ID,NAME,S1901_C01_001E,S1901_C01_001M,S1901_C02_001E,S1901_C02_001M,S1901_C03_001E,S1901_C03_001M,S1901_C04_001E,S1901_C04_001M,S1901_C01_002E,S1901_C01_002M,S1901_C02_002E,S1901_C02_002M,S1901_C03_002E,S1901_C03_002M,S1901_C04_002E,S1901_C04_002M,S1901_C01_003E,S1901_C01_003M,S1901_C02_003E,S1901_C02_003M,S1901_C03_003E,S1901_C03_003M,S1901_C04_003E,S1901_C04_003M,S1901_C01_004E,S1901_C01_004M,S1901_C02_004E,S1901_C02_004M,S1901_C03_004E,S1901_C03_004M,S1901_C04_004E,S1901_C04_004M,S1901_C01_005E,S1901_C01_005M,S1901_C02_005E,S1901_C02_005M,S1901_C03_005E,S1901_C03_005M,S1901_C04_005E,S1901_C04_005M,S1901_C01_006E,S1901_C01_006M,S1901_C02_006E,S1901_C02_006M,S1901_C03_006E,S1901_C03_006M,S1901_C04_006E,S1901_C04_006M,S1901_C01_007E,S1901_C01_007M,S1901_C02_007E,S1901_C02_007M,S1901_C03_007E,S1901_C03_007M,S1901_C04_007E,S1901_C04_007M,S1901_C01_008E,S1901_C01_008M,S1901_C02_008E,S1901_C02_008M,S1901_C03_008E,S1901_C03_008M,S1901_C04_008E,S1901_C04_008M,S1901_C01_009E,S1901_C01_009M,S1901_C02_009E,S1901_C02_009M,S1901_C03_009E,S1901_C03_009M,S1901_C04_009E,S1901_C04_009M,S1901_C01_010E,S1901_C01_010M,S1901_C02_010E,S1901_C02_010M,S1901_C03_010E,S1901_C03_010M,S1901_C04_010E,S1901_C04_010M,S1901_C01_011E,S1901_C01_011M,S1901_C02_011E,S1901_C02_011M,S1901_C03_011E,S1901_C03_011M,S1901_C04_011E,S1901_C04_011M,S1901_C01_012E,S1901_C01_012M,S1901_C02_012E,S1901_C02_012M,S1901_C03_012E,S1901_C03_012M,S1901_C04_012E,S1901_C04_012M,S1901_C01_013E,S1901_C01_013M,S1901_C02_013E,S1901_C02_013M,S1901_C03_013E,S1901_C03_013M,S1901_C04_013E,S1901_C04_013M,S1901_C01_014E,S1901_C01_014M,S1901_C02_014E,S1901_C02_014M,S1901_C03_014E,S1901_C03_014M,S1901_C04_014E,S1901_C04_014M,S1901_C01_015E,S1901_C01_015M,S1901_C02_015E,S1901_C02_015M,S1901_C03_015E,S1901_C03_015M,S1901_C04_015E,S1901_C04_015M,S1901_C01_016E,S1901_C01_016M,S1901_C02_016E,S1901_C02_016M,S1901_C03_016E,S1901_C03_016M,S1901_C04_016E,S1901_C04_016M
0,id,Geographic Area Name,Estimate!!Households!!Total,Margin of Error!!Households MOE!!Total,Estimate!!Families!!Total,Margin of Error!!Families MOE!!Total,Estimate!!Married-couple families!!Total,Margin of Error!!Married-couple families MOE!!...,Estimate!!Nonfamily households!!Total,Margin of Error!!Nonfamily households MOE!!Total,"Estimate!!Households!!Total!!Less than $10,000",Margin of Error!!Households MOE!!Total!!Less t...,"Estimate!!Families!!Total!!Less than $10,000",Margin of Error!!Families MOE!!Total!!Less tha...,Estimate!!Married-couple families!!Total!!Less...,Margin of Error!!Married-couple families MOE!!...,Estimate!!Nonfamily households!!Total!!Less th...,Margin of Error!!Nonfamily households MOE!!Tot...,"Estimate!!Households!!Total!!$10,000 to $14,999","Margin of Error!!Households MOE!!Total!!$10,00...","Estimate!!Families!!Total!!$10,000 to $14,999","Margin of Error!!Families MOE!!Total!!$10,000 ...","Estimate!!Married-couple families!!Total!!$10,...",Margin of Error!!Married-couple families MOE!!...,"Estimate!!Nonfamily households!!Total!!$10,000...",Margin of Error!!Nonfamily households MOE!!Tot...,"Estimate!!Households!!Total!!$15,000 to $24,999","Margin of Error!!Households MOE!!Total!!$15,00...","Estimate!!Families!!Total!!$15,000 to $24,999","Margin of Error!!Families MOE!!Total!!$15,000 ...","Estimate!!Married-couple families!!Total!!$15,...",Margin of Error!!Married-couple families MOE!!...,"Estimate!!Nonfamily households!!Total!!$15,000...",Margin of Error!!Nonfamily households MOE!!Tot...,"Estimate!!Households!!Total!!$25,000 to $34,999","Margin of Error!!Households MOE!!Total!!$25,00...","Estimate!!Families!!Total!!$25,000 to $34,999","Margin of Error!!Families MOE!!Total!!$25,000 ...","Estimate!!Married-couple families!!Total!!$25,...",Margin of Error!!Married-couple families MOE!!...,"Estimate!!Nonfamily households!!Total!!$25,000...",Margin of Error!!Nonfamily households MOE!!Tot...,"Estimate!!Households!!Total!!$35,000 to $49,999","Margin of Error!!Households MOE!!Total!!$35,00...","Estimate!!Famili

In [10]:
# viewing all column names to choose which ones to keep
pd.set_option('display.max_rows', None)
display(income.iloc[0])
pd.reset_option('display.max_rows')

GEO_ID                                                           id
NAME                                           Geographic Area Name
S1901_C01_001E                          Estimate!!Households!!Total
S1901_C01_001M               Margin of Error!!Households MOE!!Total
S1901_C02_001E                            Estimate!!Families!!Total
S1901_C02_001M                 Margin of Error!!Families MOE!!Total
S1901_C03_001E             Estimate!!Married-couple families!!Total
S1901_C03_001M    Margin of Error!!Married-couple families MOE!!...
S1901_C04_001E                Estimate!!Nonfamily households!!Total
S1901_C04_001M     Margin of Error!!Nonfamily households MOE!!Total
S1901_C01_002E       Estimate!!Households!!Total!!Less than $10,000
S1901_C01_002M    Margin of Error!!Households MOE!!Total!!Less t...
S1901_C02_002E         Estimate!!Families!!Total!!Less than $10,000
S1901_C02_002M    Margin of Error!!Families MOE!!Total!!Less tha...
S1901_C03_002E    Estimate!!Married-couple famil

In [11]:
usecols=['NAME','S1901_C01_001E','S1901_C01_002E', 'S1901_C01_003E', 'S1901_C01_004E']
income = income[usecols]
income

,NAME,S1901_C01_001E,S1901_C01_002E,S1901_C01_003E,S1901_C01_004E
0,Geographic Area Name,Estimate!!Households!!Total,"Estimate!!Households!!Total!!Less than $10,000","Estimate!!Households!!Total!!$10,000 to $14,999","Estimate!!Households!!Total!!$15,000 to $24,999"
1,United States,119730128,6.3,4.6,9.3
2,"Bronx County, New York",499728,13.5,9.9,13.2
3,"Kings County, New York",950856,9.3,6.9,10.0
4,"New York County, New York",758133,8.2,5.6,7.6
5,"Queens County, New York",779234,5.9,4.3,8.7
6,"Richmond County, New York",166152,8.1,3.8,6.9


In [12]:
#little more cleaning
income.columns = ['NAME','total-households','p-less-10k', 'p-10-15', 'p-15-25']
income = income.drop(index = 0, axis = 0)
income

,NAME,total-households,p-less-10k,p-10-15,p-15-25
1,United States,119730128,6.3,4.6,9.3
2,"Bronx County, New York",499728,13.5,9.9,13.2
3,"Kings County, New York",950856,9.3,6.9,10.0
4,"New York County, New York",758133,8.2,5.6,7.6
5,"Queens County, New York",779234,5.9,4.3,8.7
6,"Richmond County, New York",166152,8.1,3.8,6.9


In [13]:
# ['NAME','B25070_001E','B25070_011E', 'B25070_002E', 'B25070_009E','B25070_010E']
# ['NAME','total','total-not-comp', 'less-than-10', '40-49', 'over-50']

In [14]:
# subset of the rent table needed for calculations

rent_sub = pd.read_csv('../data/rent-as-percentage-income/ACSDT5Y2018.B25070_data_with_overlays_2020-11-19T201947.csv',
                   low_memory=False, skiprows=[1],
                   usecols=['NAME','B25070_001E','B25070_011E',
                            'B25070_002E', 'B25070_009E','B25070_010E'])
rent_sub.head()

,NAME,B25070_001E,B25070_002E,B25070_009E,B25070_010E,B25070_011E
0,"Census Tract 37, New York County, New York",789,88,41,127,61
1,"Census Tract 86.01, New York County, New York",1001,102,68,204,49
2,"Census Tract 44, New York County, New York",8145,477,821,1439,224
3,"Census Tract 86.03, New York County, New York",736,135,62,204,16
4,"Census Tract 86.02, New York County, New York",0,0,0,0,0


In [15]:
# Extract County names
rent_sub['County'] = rent_sub['NAME'].str.split(pat = ',',expand = True)[1]
rent_sub.drop(['NAME'], axis=1, inplace = True)

rent_sub

,B25070_001E,B25070_002E,B25070_009E,B25070_010E,B25070_011E,County
0,789,88,41,127,61,New York County
1,1001,102,68,204,49,New York County
2,8145,477,821,1439,224,New York County
3,736,135,62,204,16,New York County
4,0,0,0,0,0,New York County
5,1999,423,87,252,55,New York County
6,3679,69,452,1038,172,New York County
7,371,71,24,113,18,New York County
8,2901,151,308,731,76,New York County
9,1303,113,142,248,40,New York County


In [16]:
# aggregate over counties
rent_sub = rent_sub.groupby(['County']).sum()
rent_sub

,B25070_001E,B25070_002E,B25070_009E,B25070_010E,B25070_011E
County,,,,,
Bronx County,401745,12152,36347,132931,19630
Kings County,665526,28114,54147,193135,33830
New York County,575184,51242,43667,123027,25813
Queens County,431495,13971,38790,120943,24547
Richmond County,50313,1818,3425,14686,5613


In [17]:
rent_sub.columns = ['total','total-not-comp', 'less-than-10', '40-49', 'over-50']
rent_sub

,total,total-not-comp,less-than-10,40-49,over-50
County,,,,,
Bronx County,401745,12152,36347,132931,19630
Kings County,665526,28114,54147,193135,33830
New York County,575184,51242,43667,123027,25813
Queens County,431495,13971,38790,120943,24547
Richmond County,50313,1818,3425,14686,5613


In [18]:
income

,NAME,total-households,p-less-10k,p-10-15,p-15-25
1,United States,119730128,6.3,4.6,9.3
2,"Bronx County, New York",499728,13.5,9.9,13.2
3,"Kings County, New York",950856,9.3,6.9,10.0
4,"New York County, New York",758133,8.2,5.6,7.6
5,"Queens County, New York",779234,5.9,4.3,8.7
6,"Richmond County, New York",166152,8.1,3.8,6.9


In [19]:
#FINAL TABLE


### Indicator 3 - Unemployment rate

In [20]:
employment = pd.read_csv('../data/employment_ACSDP03_2018.csv',
                         low_memory=False, skiprows=[1],
                         usecols=['GEO_ID','NAME','DP03_0003E','DP03_0005E','DP03_0037E'])

In [21]:
employment.head()

,GEO_ID,NAME,DP03_0003E,DP03_0005E,DP03_0037E
0,1400000US36005000100,"Census Tract 1, Bronx County, New York",0,0,0
1,1400000US36005000200,"Census Tract 2, Bronx County, New York",1873,295,57
2,1400000US36005000400,"Census Tract 4, Bronx County, New York",3052,244,207
3,1400000US36005001600,"Census Tract 16, Bronx County, New York",2477,164,306
4,1400000US36005001900,"Census Tract 19, Bronx County, New York",1451,192,103


In [22]:
employment['DP03_0003E']=employment['DP03_0003E'].astype(float)
employment['DP03_0005E']=employment['DP03_0005E'].astype(float)

In [23]:
employment['unemployment rate'] = (100 * employment['DP03_0005E'] / employment['DP03_0003E']).round(decimals=2)
employment.head()

,GEO_ID,NAME,DP03_0003E,DP03_0005E,DP03_0037E,unemployment rate
0,1400000US36005000100,"Census Tract 1, Bronx County, New York",0.0,0.0,0,NaN
1,1400000US36005000200,"Census Tract 2, Bronx County, New York",1873.0,295.0,57,15.75
2,1400000US36005000400,"Census Tract 4, Bronx County, New York",3052.0,244.0,207,7.99
3,1400000US36005001600,"Census Tract 16, Bronx County, New York",2477.0,164.0,306,6.62
4,1400000US36005001900,"Census Tract 19, Bronx County, New York",1451.0,192.0,103,13.23


### Indicator 4 - employment in impacted sectors (retail sales, hospitality service, personal care)

### Indicator 5 - CARES act exclusion

In [24]:
# Import and clean the information about total population from ACS
dp05 = pd.read_csv("../data/cares-exclusion/ACSDP5Y2018.DP05_data_with_overlays_2020-11-20T095617.csv",
                   usecols = ["GEO_ID","NAME","DP05_0001E"], skiprows=[1])
dp05.rename(columns = {"DP05_0001E" : "total_population"}, inplace = True)

# Extract County names
dp05['County'] = dp05['NAME'].str.split(pat = ',',expand = True)[1]
dp05.head()

,GEO_ID,NAME,total_population,County
0,1400000US36061000100,"Census Tract 1, New York County, New York",0,New York County
1,1400000US36061000201,"Census Tract 2.01, New York County, New York",2835,New York County
2,1400000US36061000202,"Census Tract 2.02, New York County, New York",7764,New York County
3,1400000US36061000500,"Census Tract 5, New York County, New York",0,New York County
4,1400000US36061000600,"Census Tract 6, New York County, New York",9731,New York County


In [25]:
# Import data about population that's in IRS' system
irs = pd.read_csv('../data/cares-exclusion/18zp33ny.csv', skiprows=[0, 1, 2, 4, 5], 
                       usecols = ["ZIP\ncode [1]", "Size of adjusted gross income", 
                                  "Number of returns", "Number of single returns", 
                                  "Number of joint returns", "Number of head of household returns", 
                                  "Number of \nindividuals [2]", "Number of dependents"], 
                  dtype = np.str)

irs.rename(columns={"ZIP\ncode [1]": "ZIP", "Number of \nindividuals [2]": "Number of individuals"}, 
           inplace = True)

irs.head(15)

,ZIP,Size of adjusted gross income,Number of returns,Number of single returns,Number of joint returns,Number of head of household returns,Number of individuals,Number of dependents
0,00000,Total,"9,632,130","5,003,730","2,947,270","1,460,570","17,260,550","5,256,960"
1,00000,"$1 under $25,000","3,161,790","2,230,890","384,320","496,390","4,289,760","1,269,570"
2,00000,"$25,000 under $50,000","2,223,030","1,239,350","433,380","499,300","3,878,510","1,246,870"
3,00000,"$50,000 under $75,000","1,365,250","709,530","379,140","236,500","2,417,530","680,400"
4,00000,"$75,000 under $100,000","871,610","353,000","378,620","110,550","1,726,310","476,400"
5,00000,"$100,000 under $200,000","1,376,380","362,150","879,400","102,230","3,245,060","993,140"
6,00000,"$200,000 or more","634,070","108,810","492,410","15,600","1,703,380","590,580"
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10001,NaN,"16,150","11,740","2,800","1,160","21,760","3,350"
9,10001,"$1 under $25,000","3,680","2,900",300,370,"4,390",800


In [26]:
irs_zip = irs[irs["ZIP"].notna() & irs["Size of adjusted gross income"].isna() & irs["Number of returns"].notna()].copy()

for col in irs_zip.columns[2:]:
    irs_zip[col] = irs_zip[col].str.replace(",", "").astype(int) 

irs_zip.head(10)

,ZIP,Size of adjusted gross income,Number of returns,Number of single returns,Number of joint returns,Number of head of household returns,Number of individuals,Number of dependents
8,10001,NaN,16150,11740,2800,1160,21760,3350
16,10002,NaN,42400,26260,9900,5360,67080,16200
24,10003,NaN,29270,22200,5490,870,38080,4650
32,10004,NaN,2470,1450,820,110,4280,1100
40,10005,NaN,6160,4480,1290,170,8280,1200
48,10006,NaN,2510,1840,480,100,3450,550
56,10007,NaN,3760,2160,1360,140,6520,1720
64,10009,NaN,33200,23870,5160,3410,45840,8850
72,10010,NaN,17400,12510,3760,690,24130,3590
80,10011,NaN,30370,22340,5920,1260,40580,5510


In [27]:
# Import the crosstalk file linking zip code to census track
zip_to_track = pd.read_csv("../data/cares-exclusion/ZIP_TRACT_122018.csv", dtype = {"zip":str, "tract":str})
zip_to_track.head()

,zip,tract,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,00601,72001956700,0.671739,0.411079,0.530973,0.654116
1,00603,72005401302,0.082247,0.036098,0.072491,0.079523
2,00623,72023830400,0.331718,0.551763,0.479730,0.344927
3,00638,72039955700,0.436658,0.819444,0.777778,0.455135
4,00646,72051540100,0.142668,0.342226,0.148515,0.156662


In [28]:
# Join the irs data with zip to track data
irs_zt = zip_to_track.join(irs_zip[["ZIP", "Number of individuals"]].set_index("ZIP"), on = "zip", how = "left")
irs_zt = irs_zt[irs_zt["Number of individuals"].notna()]
irs_zt.head()

,zip,tract,res_ratio,bus_ratio,oth_ratio,tot_ratio,Number of individuals
7891,10001,36061009500,0.089990,0.230449,0.226739,0.149923,21760.0
7892,10001,36061009100,0.099933,0.056752,0.075440,0.082844,21760.0
7893,10002,36061002201,0.085143,0.035253,0.063872,0.080472,67080.0
7894,10001,36061010300,0.125852,0.030322,0.047360,0.086195,21760.0
7895,10013,36061003300,0.233373,0.208787,0.218345,0.225716,24330.0


In [29]:
# Summarize the irs data weighted by ratio of zip codes in each census track
irs_zt["N_corrected"] = irs_zt["tot_ratio"] * irs_zt["Number of individuals"]
irs_by_tract = irs_zt[["tract", "N_corrected"]].groupby("tract").sum()
irs_by_tract.head()

,N_corrected
tract,
36001000100,1468.305674
36001000200,2951.837908
36001000300,4900.253672
36001000401,2350.772770
36001000403,3067.276198


In [30]:
# Join the irs data with the ACS data
dp05["tract"] = [i[9:] for i in dp05["GEO_ID"]]
dp05_irs = dp05.join(irs_by_tract, on = "tract")
dp05_irs["CARES_exclusion"] = 1 - dp05_irs["N_corrected"] / dp05_irs["total_population"]
dp05_irs.head()
# ? What do we want to do about the tracts where the total_population is zero or there is no data from the irs?

,GEO_ID,NAME,total_population,County,tract,N_corrected,CARES_exclusion
0,1400000US36061000100,"Census Tract 1, New York County, New York",0,New York County,36061000100,3.679345,-inf
1,1400000US36061000201,"Census Tract 2.01, New York County, New York",2835,New York County,36061000201,1352.964425,0.522764
2,1400000US36061000202,"Census Tract 2.02, New York County, New York",7764,New York County,36061000202,6075.198137,0.217517
3,1400000US36061000500,"Census Tract 5, New York County, New York",0,New York County,36061000500,NaN,NaN
4,1400000US36061000600,"Census Tract 6, New York County, New York",9731,New York County,36061000600,8625.930297,0.113562


In [31]:
# Check if the estimates make sense
# the proportion of population not covered readily by the CARES Act in New York City
print("Number of people not readily covered by the CARES Act in New York City:")
print(dp05_irs.sum()["total_population"] - dp05_irs.sum()["N_corrected"])
print("Number of undocumented immigrants based on a report from Migration Policy Institute")
# https://www.migrationpolicy.org/data/unauthorized-immigrant-population/state/NY
print(866000)
# These are on the same magnitude
print("Proportion of population")
print(100 - 100* dp05_irs.sum()["N_corrected"] / dp05_irs.sum()["total_population"], "%")
# Similar to results from the LA data

Number of people not readily covered by the CARES Act in New York City:
1192894.275184908
Number of undocumented immigrants based on a report from Migration Policy Institute
866000
Proportion of population
14.127603285247943 %


In [32]:
print("Total number of tracks", dp05_irs.shape[0])
print(dp05_irs[dp05_irs["CARES_exclusion"] < 0].shape[0], "of them have higher IRS counts than ACS counts")
dp05_irs[(-10000 < dp05_irs["CARES_exclusion"]) & (dp05_irs["CARES_exclusion"] < 0)].sort_values("CARES_exclusion").head(20)
# Notice most of these census tracks don't have a lot of residents

Total number of tracks 2167
488 of them have higher IRS counts than ACS counts


,GEO_ID,NAME,total_population,County,tract,N_corrected,CARES_exclusion
93,1400000US36061009400,"Census Tract 94, New York County, New York",54,New York County,36061009400,1813.903374,-32.590803
95,1400000US36061009600,"Census Tract 96, New York County, New York",141,New York County,36061009600,4555.559043,-31.308929
113,1400000US36061011300,"Census Tract 113, New York County, New York",110,New York County,36061011300,2547.039358,-22.154903
2143,1400000US36085022800,"Census Tract 228, Richmond County, New York",12,Richmond County,36085022800,247.350600,-19.612550
101,1400000US36061010200,"Census Tract 102, New York County, New York",100,New York County,36061010200,1962.359705,-18.623597
107,1400000US36061010900,"Census Tract 109, New York County, New York",182,New York County,36061010900,2336.994895,-11.840631
142,1400000US36061014300,"Census Tract 143, New York County, New York",3,New York County,36061014300,20.773382,-5.924461
1184,1400000US36047070202,"Census Tract 702.02, Kings County, New York",24,Kings County,36047070202,154.715892,-5.446495
430,1400000US36005021900,"Census Tract 219, Bronx County, New York",1045,Bronx County,36005021900,4466.141542,-3.273820
223,1400000US36061021703,"Census Tract 217.03, New York County, New York",8,New York County,36061021703,28.034786,-2.504348


In [33]:
# Make the negative values zero for the metric
dp05_irs["CARES_exclusion_c"] = [i if i > 0 else 0 for i in dp05_irs["CARES_exclusion"]]
# note that this also makes all tracks with zero population have a CARES_exclusion of zero, which is argueable
dp05_irs.head()

,GEO_ID,NAME,total_population,County,tract,N_corrected,CARES_exclusion,CARES_exclusion_c
0,1400000US36061000100,"Census Tract 1, New York County, New York",0,New York County,36061000100,3.679345,-inf,0.000000
1,1400000US36061000201,"Census Tract 2.01, New York County, New York",2835,New York County,36061000201,1352.964425,0.522764,0.522764
2,1400000US36061000202,"Census Tract 2.02, New York County, New York",7764,New York County,36061000202,6075.198137,0.217517,0.217517
3,1400000US36061000500,"Census Tract 5, New York County, New York",0,New York County,36061000500,NaN,NaN,0.000000
4,1400000US36061000600,"Census Tract 6, New York County, New York",9731,New York County,36061000600,8625.930297,0.113562,0.113562
